In [0]:
from functools import partial
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

if not "ffmpeg" in animation.writers.list():
  #http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
  print("ffmpeg not found, installing:")
  !apt-get -qq install -y ffmpeg


# get ffmpeg path with !which ffmpeg
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
        
# set videos to automatically show in notebook
plt.rcParams['animation.html'] = 'html5'

# consider adding the imagemagick backed and using gifs

In [0]:
def rk4_step(dxdt, t, x, h):
    
        k1 = dxdt(t, x)
        t1 = t + h/2;
        x1 = x + h*k1/2;
        
        k2 = dxdt(t1, x1)
        x2 = x + h*k2/2;
        t2 = t + h/2;
        
        k3 = dxdt(t2, x2)
        x3 = x + h*k3;
        t3 = t + h;
        
        k4 = dxdt(t3, x3)
        
        return tf.group(
            tf.assign_add(x, h/6 * (k1 + k2*2 + k3*2 + k4)),
            tf.assign_add(t, h),
            name='update_state'
        )


In [0]:
def pendulum_hamiltonian(qval,p):

    T = tf.reduce_sum(1/2 * p**2) # = 1/2 mv^2
    V = -1e-2 *tf.cos(qval[:,0])  # = mgh = mgcos(theta)
    
    return T + V


In [0]:
def hamiltonian_time_derivative(H, t, x):

      q, p = tf.split(x, 2, axis=1)

      dpdt = tf.gradients(-H(q, p), q)
      dqdt = tf.gradients( H(q, p), p)

      #TODO This bit isn't clear
      dxdt = tf.concat([dqdt,dpdt],axis=2)[0]
            
      return dxdt


In [0]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [0]:
# setup simulation
pend_length = 10;
h = tf.constant(.05, dtype=tf.float32, name='time_step');
x0 = np.zeros((1,2))

# Initial Condition, x = [0, tf.concat[q,p]]
x0[:,0] += np.pi - 1e-2 #q
x0[:,1] += 0       #qdot



In [0]:
x = tf.Variable(x0, dtype=tf.float32)
t = tf.Variable(0, dtype=tf.float32)

dxdt = partial(hamiltonian_time_derivative, pendulum_hamiltonian)

update = rk4_step(dxdt, t, x, h)

In [0]:
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter('logdir/', sess.graph)

In [0]:
N = 4000
state = np.array([x0]*N, dtype=np.float32)

# run simulation
for i in range(N):
    _, state[i] = sess.run([update,x]) #output [timestep (N), first column of x (blank), second column of x which is q or qdot]

summary_writer.close()

In [220]:
# plot
import seaborn
fig = plt.figure(figsize=(8,4))

#plt.title('Pendulum')


fig, (ax,ax2) = plt.subplots(
    ncols=2, 
    sharey=True,
    gridspec_kw={
        "width_ratios":[4,4],
    }
)

lim = 1.2*pend_length
lims = (-lim, lim)

ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect('equal')
ax.grid()
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")

hs = -pend_length*np.cos(state[:,0,0])
ax2.plot(hs, label=r"$\theta$")
ax2.plot(state[:,0,1], label=r"$\dot\theta$")
ax2.set_xlabel(r'$t$')

ax2.set_aspect("auto")

ax2.legend(loc="upper right")

timeline, = ax2.plot([],[], c="red")

line, = ax.plot([], [], 'o-', lw=2)

time_template = r'time = {}'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


plt.tight_layout()

skip = 50

def init():
    line.set_data([], [])
    
    time_text.set_text('')
    timeline.set_xdata([i,i])
    timeline.set_ydata([-pend_length,pend_length])
    
    return line, time_text, timeline


def animate(i):
    theta = state[i,0,0]
    line.set_data(
        [0, pend_length*np.sin(theta)],
        [0, -pend_length*np.cos(theta)]
    )
    timeline.set_xdata([i,i])
    time_text.set_text(time_template.format(i))
    return line, time_text, timeline
  
plt.close()

ani = animation.FuncAnimation(fig, animate, np.arange(1, N, skip),
                              interval=50, blit=True, init_func=init)


In [221]:
ani